In [ ]:
import requests
import json
import pandas as pd
import numpy as np
from typing import Tuple
from pandas import DataFrame

from sklearn.neighbors import BallTree

from time import sleep
import os

import plotly.express as px
import h3

from plotly.subplots import make_subplots
import plotly.graph_objects as go



In [ ]:

def get_orgs_yamaps(super_cat: str,
                    sub_cat: str,
                    skip: int,
                    bbox: str='37.521587,55.707375~37.712390,55.796668',
                   ) -> Tuple[DataFrame, bool]:
    
    # make api request
    ymaps_api = '5b654382-731c-4c23-80eb-f3c24084460d'
    
    request_url = (
        f'https://search-maps.yandex.ru/v1/'
        f'?apikey={ymaps_api}'
        f'&text={sub_cat}'
        f'&lang=ru_RU'
        f'&type=biz'
        f'&bbox={bbox}'
        f'&rspn=1'
        f'&skip={skip}'
        f'&results=500'
    )
    
    resp = requests.get(request_url)
    print(sub_cat, skip, 'resp_code', resp.status_code)

    # extract valid data from json
    source = []
    data = resp.json()
    features = data.get('features')
    if features:
        print(sub_cat, skip, 'count', len(features))
        
        for feature in features:
            geo_point = feature.get('geometry', {}).get('coordinates', [])
            ya_id = feature.get('properties', {}).get('CompanyMetaData', {}).get('id', None)
            name = feature.get('properties', {}).get('CompanyMetaData', {}).get('name', None)
            address = feature.get('properties', {}).get('CompanyMetaData', {}).get('address', None)
            url = feature.get('properties', {}).get('CompanyMetaData', {}).get('url', None)

            phone_arr = [
                ph.get('formatted')
                for ph in feature.get('properties', {}).get('CompanyMetaData', {}).get('Phones', [{}])
                if ph
            ]

            class_arr = [
                cl.get('class')
                for cl in feature.get('properties', {}).get('CompanyMetaData', {}).get('Categories', [{}])
                if cl
            ]

            categories_arr = [
                ct.get('name')
                for ct in feature.get('properties', {}).get('CompanyMetaData', {}).get('Categories', [{}])
                if ct
            ]

            hours_text = feature.get('properties', {}).get('CompanyMetaData', {}).get('Hours', {}).get('text', None)

            temp = (
                super_cat,
                sub_cat,
                geo_point,
                ya_id,
                name,
                address,
                url,
                phone_arr,
                class_arr,
                categories_arr,
                hours_text
            )
            source.append(temp)

    # convert to dataframe
    columns = [
        'super_cat',
        'sub_cat',
        'geo_point',
        'ya_id',
        'name',
        'address',
        'url',
        'phone_arr',
        'class_arr',
        'categories_arr',
        'hours_text'
    ]
    df = pd.DataFrame(source,columns=columns)
    
    # check if there is more data left to query
    found = data.get('properties',{}).get('ResponseMetaData',{}).get('SearchResponse',{}).get('found',0)
    results = data.get('properties',{}).get('ResponseMetaData',{}).get('SearchRequest',{}).get('results',0)
    skip = data.get('properties',{}).get('ResponseMetaData',{}).get('SearchRequest',{}).get('skip',0)
    
    if skip + results < found:
        is_more = True
    else:
        is_more = False
        
    return df, is_more



def get_orgs_yamaps_all(super_cat: str,
                        sub_cat: str,
                        bbox: str='37.521587,55.707375~37.712390,55.796668',
                       )-> DataFrame:
    
    skip = 0
    is_more = True
    dfs = []
    
    while is_more:
        df, is_more = get_orgs_yamaps(super_cat=super_cat,
                                      sub_cat=sub_cat,
                                      skip=skip,
                                      bbox=bbox)
        dfs.append(df)
        skip += 500
        print('skip += 500')
        
    return pd.concat(dfs)
        

def get_orgs_categories(json_path: str='bcats_raw.json'):
    with open(json_path) as data_file:
        data_loaded = json.load(data_file)
    return [(cat, sub) for cat,sub in data_loaded.items()]


    
def get_save_all_orgs(path: str='temp/'):
    
    cats = get_orgs_categories()
    
    names = []
    for super_cat in cats:
        print('getting super...', super_cat[0])
        for sub_cat in super_cat[1]:
            
            print(' getting sub...', sub_cat)
            df = get_orgs_yamaps_all(super_cat=super_cat[0], sub_cat=sub_cat)

            filename = f'{path}{super_cat[0]}_{sub_cat}.csv'
            df.to_csv(filename, index=False)
            print(' success...', filename)
            names.append(filename)
            
    return names


def read_saved(names):
    return pd.concat([pd.read_csv(name) for name in names])
    


In [ ]:

def make_coord_grid(left_bottom: list=[37.521587, 55.707375],
                    right_top: list =[37.712390, 55.796668],
                    x_count: int=100,
                    y_count: int=100):
    """
    moscow ttk: '37.521587,55.707375~37.712390,55.796668'
    
    """
    # make linearly spaced coordinate grid
    xmin, xmax, ymin, ymax = left_bottom[0], right_top[0], left_bottom[1], right_top[1]
    xx, yy = np.meshgrid(np.linspace(xmin,xmax,x_count), np.linspace(ymin,ymax,y_count))
    # make coordinate grid hexagonal by shifting each two lines
    # works when x_count = y_count = 100
    xx[::2, :] += 0.001
    
    grid = np.hstack([yy.reshape(x_count*y_count,1),
                      xx.reshape(x_count*y_count,1)])
    
    # make kdtree for effective grid search
    tree = BallTree(np.deg2rad(grid), metric='haversine')
    
    return grid, tree


def load_coord_grid(path=None):
    if path is None:
        path = os.getcwd()+'/'+'grid.csv'
    grid = pd.read_csv(path).iloc[:,1:].to_numpy()
    tree = BallTree(np.deg2rad(grid), metric='haversine')
    return grid, tree

grid, tree = load_coord_grid()



In [ ]:

def get_rent_yamaps(lat: float,
                    lon: float):
    api_url = (
        f'https://realty.yandex.ru/api/1.0/heatmap/price-rent/point/value'
        f'?latitude={lat}'
        f'&longitude={lon}'
    )
    data = requests.get(api_url).json()
    data = data.get('response', {})
    return (
        data.get('valueDescription'),
        data.get('valueFrom'),
        data.get('valueTo'),
        data.get('level'),
        data.get('maxLevel'),
        data.get('realValue')
    )


def get_rent_yamaps_all(grid: np.ndarray):
    
    source = []
    
    for idx, p in enumerate(grid):
        rent = get_rent_yamaps(lat=p[0], lon=p[1])
        print(tuple(p) + rent)
        source.append(tuple(p) + rent)
        # sleep(0.01)
    
    columns = [
        'point_index',
        'lat',
        'lon',
        'valueDescription',
        'valueFrom',
        'valueTo',
        'level',
        'maxLevel',
        'realValue'
    ]
    
    return pd.DataFrame(source, columns=columns)




In [ ]:
def create_traffic(files: list):
    for idx, file in enumerate(files):
        # read csv ~ 50 mln rows, required ~ 2.5 GB memory
        df_traffic_0 = pd.read_csv(file)
        print(idx, 'read csv')
        # ts to datetime
        df_traffic_0['ts'] = pd.to_datetime(df_traffic_0['ts'])
        print(idx, 'ts to datetime')
        # gps to lat lon floats, required ~ 20 GB memory (physical + swap)
        coords = df_traffic_0['gps'].str.split(',', expand=True)
        print(idx, 'gps to lat lon floats - split')
        coords.columns = ['lat', 'lon']
        coords['lat'] = coords['lat'].str.replace('(','').astype(float)
        print(idx, 'gps to lat lon floats - replace 1')
        coords['lon'] = coords['lon'].str.replace(')','').astype(float)
        print(idx, 'gps to lat lon floats - replace 2')
        df_traffic_0 = pd.concat([df_traffic_0[['ts', 'ap_mac', 'device_id', 'user_id']], coords], axis=1)
        print(idx, 'gps to lat lon floats - concat')
        del coords
        
        # get unique coordinates of wi-fi points 
        unique_coords = df_traffic_0[['ap_mac', 'lat', 'lon']].drop_duplicates(subset=['lat', 'lon'], keep='first')
        
        # create hour column
        df_traffic_0['ts_h'] =  df_traffic_0['ts'].dt.round('H')
        
        # match wi-fi point coordinates with our grid
        coords_matrix = unique_coords[['lat', 'lon']].to_numpy()
        dist, indexes = tree.query(np.deg2rad(coords_matrix))
        points = np.append(indexes, grid[indexes.ravel()], axis=1)
        points = np.hstack([coords_matrix, points, dist])
        points = pd.DataFrame(points, columns=['lat', 'lon', 'point_index', 'point_lat', 'point_lon', 'dist'])
        points = pd.merge(unique_coords, points, on=['lat', 'lon'], how='left')
        points = points[points['dist']<=points['dist'].quantile(.90)]
        points_join = points[['ap_mac', 'point_index']]
        # for faster join - create index and sort it
        df_traffic_0.set_index('ap_mac', inplace=True)
        df_traffic_0.sort_index(inplace=True)
        # for faster join - create index and sort it
        points_join.set_index('ap_mac', inplace=True)
        points_join.sort_index(inplace=True)
        # join
        df_traffic_0 = df_traffic_0.join(points_join, how='left')
        
        # group by grid point, hour and count devices
        df_traffic_agg = df_traffic_0.groupby(
            ['point_index', 'ts_h']
        ).agg(
            count_device=('device_id', 'count'),
            unique_device=('device_id', 'nunique')
        ).reset_index()
        
        # save results ~ 0.5 mln rows, 20 Mb
        df_traffic_agg.to_csv(f'df_traffic_agg_{idx}.csv', index=False)
        
        
def preprocess_traffic(files):
    
    df_agg_all = read_saved(files)
     
    # create day column
    df_agg_all['ts_h'] = pd.to_datetime(df_agg_all['ts_h'])
    df_agg_all['ts_d'] = df_agg_all['ts_h'].dt.round('D')
    df_agg_all['is_night'] = df_agg_all['ts_h'].apply(lambda x: 1 if ((x.hour>=22) or (x.hour<=7)) else 0)


    dfs = []

    dta = [('all', df_agg_all),
           ('night', df_agg_all[df_agg_all['is_night']==1]),
           ('day', df_agg_all[df_agg_all['is_night']==0])]

    for group, df_agg in dta:

        # groupby point and date, sum unique devices
        df_agg = df_agg.groupby(
            ['point_index', 'ts_d']
        ).agg( 
            unique_device=('unique_device', 'sum')
        ).reset_index()

        # group only by point, and count median of unique devices per point from 03-05 month
        df_agg = df_agg.groupby(
            ['point_index']
        ).agg(
            unique_device=('unique_device', 'median')
        ).reset_index()

        # add lat lon columns from grid
        df_gr = pd.DataFrame(grid).reset_index()
        df_gr.columns=['point_index', 'point_lat', 'point_lon']
        df_fin = pd.merge(df_gr, df_agg, on=['point_index'], how='left').fillna(0)

        # blur points = make every point a sum from 18 surrounding points
        blur = 18
        if blur > 0:
            indexes = tree.query(np.deg2rad(df_fin[['point_lat', 'point_lon']].to_numpy()), k=blur)[1]
            df_fin.index = df_fin['point_index'].astype(int)
            rank_matrix = np.concatenate([df_fin[['unique_device']].reindex(indexes[:,idx]).to_numpy()
                                          for idx in range(0,blur)],
                                         axis=1)
            df_fin['unique_device'] = np.sum(rank_matrix, axis=1)

        # save
        df_fin = df_fin.reset_index(drop=True)
        df_fin.columns = ['point_index', 'point_lat', 'point_lon', 'metric_traffic']
        df_fin.insert(0, 'group', group)
        dfs.append(df_fin)

    dta = pd.concat(dfs)
    dta.to_csv('metric_traffic.csv', index=False)
    return None
        

In [ ]:

def create_competition():
    path = os.getcwd()+'/temp/'
    # PARSING, takes time
    get_save_all_orgs(path)
    
    names = [path+name 
             for name in os.listdir(path) if '.csv' in name]

    df = read_saved(names)
    df['lat'] = df['geo_point'].apply(lambda x: json.loads(x)[1])
    df['long'] = df['geo_point'].apply(lambda x: json.loads(x)[0])
    df.to_csv('competition_data.csv', index=False)
    
    return os.getcwd()+'/'+'competition_data.csv'
    

def create_metric_rent():
    # PARSING, takes time
    df_rent = get_rent_yamaps_all(grid)
    
    df_rt = df_rent[~df_rent['realValue'].isna()][['lat', 'lon', 'realValue']]
    df_rt.rename(columns={'realValue': 'metric_rent'}, inplace=True)
    df_rt.to_csv('metric_rent.csv', index=False)
    return os.getcwd()+'/'+'metric_rent.csv'


def haversine_exc(point_1, point_2=(55.7525, 37.6231)):
    lon1, lat1, lon2, lat2 = map(np.radians, [point_1[1], point_1[0], point_2[1], point_2[0]])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    return 6367 * 2 * np.arcsin(np.sqrt(a))


In [ ]:

def match_with_grid_points(df, columns=['lat', 'long']):
    indexes = tree.query(np.deg2rad(df[columns].to_numpy()))[1]
    points = np.append(indexes, grid[indexes.ravel()], axis=1)
    return pd.concat([df,
                      pd.DataFrame(points, columns=['point_index', 'point_lat', 'point_lon'])],
                     axis=1)

def preprocess_data():
    # read
    competition_data = pd.read_csv('competition_data.csv')
    metric_rent = pd.read_csv('metric_rent.csv')
    metric_traffic = pd.read_csv('metric_traffic.csv')
    
    # all dfs would be mapped to points
    metric_rent = metric_rent.reset_index().rename(columns={'index':'point_index',
                                             'lat': 'point_lat',
                                             'lon': 'point_lon'})

    metric_traffic = metric_traffic.reset_index().rename(columns={'index':'point_index',
                                             'lat': 'point_lat',
                                             'lon': 'point_lon'})
    
    competition_data = match_with_grid_points(competition_data,
                                              columns=['lat', 'long'])
    
    competition_data.to_csv('competition_data.csv', index=False)
    metric_rent.to_csv('metric_rent.csv', index=False)
    metric_traffic.to_csv('metric_traffic.csv', index=False)
    metric_rent[['point_index', 'point_lat', 'point_lon']].to_csv('grid.csv', index=False)
    
    return [os.getcwd()+'/'+'competition_data.csv', os.getcwd()+'/'+'metric_rent.csv',\
            os.getcwd()+'/'+'metric_traffic.csv', os.getcwd()+'/'+'grid.csv']
    


In [ ]:
def point_to_hex(lat, lon):
    
    coord_matrix = [
        [cords[1], cords[0]]
        for cords in h3.h3_to_geo_boundary(h3.geo_to_h3(lat, lon, 10))
    ]

    return coord_matrix + [coord_matrix[0]]


def df_to_geojson(df, top=10000):
    geojson = {
        'type': 'FeatureCollection',
        'features': [
            {'type': 'Feature',
             'geometry': {
                 'type': 'Polygon',
                 'coordinates': [
                     point_to_hex(row.point_lat, row.point_lon)
                 ]},
             'properties': {
                 'point_index': row.point_index,
                 'point_lat': row.point_lat,
                 'point_lon': row.point_lon,
                 'metric_rent': row.metric_rent,
                 'metric_traffic': row.metric_traffic,
                 'dist': row.dist,
                 'count_comps': row.count_comps,
                 'rank_rent': row.rank_rent,
                 'rank_traffic': row.rank_traffic,
                 'rank_comps': row.rank_comps,
                 'total_rank': row.total_rank
             },
             'id': row.point_index
            }
            for idx, row in enumerate(df.itertuples()) if idx <= top
        ]
    }
    return geojson

def geojson_to_df(geojson):
    return pd.DataFrame([feature['properties'] for feature in geojson['features']])



In [ ]:


competition_data = pd.read_csv('competition_data.csv')
competition_data['comps_array'] = np.array(competition_data.to_dict('records'))

metric_rent = pd.read_csv('metric_rent.csv')
metric_rent['dist'] = metric_rent.apply(lambda x: haversine_exc([x['point_lat'], x['point_lon']]), axis=1)

metric_traffic = pd.read_csv('metric_traffic.csv')


def find_optimal(sub_cat,
                 imp_rent,
                 imp_traffic,
                 imp_comps,
                 center_dist,
                 top,
                 blur,
                 hours):
    
    # leave only certain hours traffic data
    df_ranked = metric_traffic[metric_traffic['group']==hours].iloc[:, 1:]
    
    # merge with rent data
    df_ranked = pd.merge(metric_rent, df_ranked[['point_index', 'metric_traffic']],
                         on='point_index', how='left')

    # leave only certain points, that are X km far from city center
    df_ranked = df_ranked[df_ranked['dist'] < center_dist]
    
    # df_ranked = df_ranked[df_ranked['metric_traffic']>0]
    
    # leave only certain busnesses here
    count_comps = competition_data[
        competition_data['sub_cat'].isin(sub_cat)
    ][
        ['point_index', 'ya_id', 'comps_array']
    ].groupby(
        'point_index'
    ).agg({
        'ya_id': 'nunique',
        'comps_array': list
    }).reset_index().rename(columns={'ya_id': 'count_comps'})

    df_ranked = pd.merge(df_ranked, count_comps, on='point_index', how='left').fillna(0)
    
    # apply blur to count comps
    if blur > 0:
        indexes = tree.query(np.deg2rad(df_ranked[['point_lat', 'point_lon']].to_numpy()), k=blur)[1]
        
        df_ranked.index = df_ranked['point_index'].astype(int)
        
        rank_matrix = np.concatenate([df_ranked[['count_comps']].reindex(indexes[:,idx]).to_numpy()
                                      for idx in range(0,blur)],
                                     axis=1)
        
        df_ranked['count_comps'] = np.sum(rank_matrix, axis=1)
    

    # compute weighted rating
    tot = sum([imp_rent, imp_traffic, imp_comps])
    w_r, w_t, w_c = imp_rent/tot, imp_traffic/tot, imp_comps/tot

    df_ranked['rank_rent'] = (1/(df_ranked['metric_rent']+1)).rank(pct=True)
    df_ranked['rank_traffic'] = df_ranked['metric_traffic'].rank(pct=True)
    df_ranked['rank_comps'] = (1/(df_ranked['count_comps']+1)).rank(pct=True)
        
    
    
    df_ranked['total_rank'] = ( w_r * df_ranked['rank_rent'] \
                              + w_t * df_ranked['rank_traffic'] \
                              + w_c * df_ranked['rank_comps'] ).round(5)
    

    df_ranked = df_ranked.sort_values(by='total_rank', ascending=False)[:top]
    
    # convert to geojson
    return df_to_geojson(df_ranked)
    
print('ok')


In [ ]:
# top_lat = 55.774497
# bottom_lat = 55.728464

# top_long = 37.660644
# bottom_long = 37.582188

competition_data.groupby('sub_cat')['ya_id'].count().sort_values(ascending=False).reset_index()['sub_cat'].unique()

In [ ]:
joury_mapping = {
    "Фуд ритейл": ['Магазин продуктов', 'Супермаркет', 'Магазин овощей и фруктов', 'Гипермаркет', 'Продуктовый гипермаркет'],
    "Парикмахерские, салоны красоты и т.п.": ['Салон красоты', 'Парикмахерская', 'Барбершоп'],
    "Рестораны, кафе, бары и ночные клубы": ['Ресторан', 'Кафе', 'Ночной клуб', 'Бар, паб'],
    "Бытовая химия, магазины косметики и т.п.": ['Магазин хозтоваров и бытовой химии'],
    "Клиники, стоматологии, вет.клиники": ['Медцентр', 'Поликлиника', 'Стоматологическая клиника', 'Ветеринарная клиника'],
    "Услуги (кофе на вынос, ремонт обуви и ювелирных изделий) и мелкая торговля (фото, пресса, мороженое).": [
        'Ремонт одежды',
        'Ремонт обуви',
        'Ремонт часов',
        'Ремонт бытовой техники',
        'Мороженое',
        'Кофейня',
        'Фотомагазин',
        'Магазин смешанных товаров',
        'Магазин чая и кофе'
    ]
}

joury_mapping

1. Фуд ритейл
2. Парикмахерские, салоны красоты и т.п.
3. Рестораны, кафе, бары и ночные клубы
4. Бытовая химия, магазины косметики и т.п.
5. Клиники, стоматологии, вет.клиники
6. Услуги (кофе на вынос, ремонт обуви и ювелирных изделий) и мелкая торговля (фото, пресса, мороженое).

In [ ]:
map_box_token = 'pk.eyJ1IjoiYXNoZXJhc2hlciIsImEiOiJja3BncjV0d28yZHVvMnBsbGx5aGp4d3lrIn0.sqhH34AnR9FQCC2tysD3vg'
px.set_mapbox_access_token(map_box_token)

sub_cat = ['Магазин продуктов']#joury_mapping['Парикмахерские, салоны красоты и т.п.']
imp_rent = 0
imp_traffic= 0
imp_comps = 10
center_dist = 12.2
top = 100000
blur = 18
hours='night'

geojs = find_optimal(sub_cat,
                     imp_rent,
                     imp_traffic,
                     imp_comps,
                     center_dist,
                     top,
                     blur,
                     hours)

fig = px.choropleth_mapbox(geojson_to_df(geojs),
                           geojson=geojs,
                           color="total_rank",
                           locations="point_index",
                           opacity=0.8,
                           color_continuous_scale="plotly3_r",
                           zoom=11.0,
                           center={
                               "lat": 55.7525,
                               "lon": 37.6231
                           },
                           hover_data=[
                               'point_index',
                               'point_lat',
                               'point_lon',
                               'metric_rent',
                               'metric_traffic',
                               'count_comps',
                               
                               'total_rank'
                           ]
                          )



fig.update_layout(margin=dict(b=0, t=0, l=0, r=0))
name = '_'.join(sub_cat) + f'_top={top}_blur={blur}_wrnt={imp_rent}_wtrf={imp_traffic}__wcmp={imp_comps}'
print(name)
fig.show()

In [ ]:

with open('geojs.json', 'w') as f:
    json.dump(geojs, f)

In [ ]:
fig.write_image(f'{name}.png')


In [ ]:

def create_traffic_hours_ts(files):
    df_agg = read_saved(files)

    # create day column
    df_agg['ts_h'] = pd.to_datetime(df_agg['ts_h'])
    df_agg['hour'] = df_agg['ts_h'].dt.hour
    # df_agg['weekday'] = df_agg['ts_h'].dt.weekday


    df_agg = df_agg.groupby(['point_index', 'hour'])['unique_device'].median().reset_index()


    # add lat lon columns from grid
    df_gr = pd.DataFrame(grid).reset_index()
    df_gr.columns=['point_index', 'point_lat', 'point_lon']
    df_fin = pd.merge(df_gr, df_agg, on=['point_index'], how='left').fillna(0)

    # pivot - unpivot to fill missing hour values
    df_fin = pd.pivot_table(df_fin,
                            values='unique_device',
                            index='point_index',
                            columns='hour',
                            fill_value=0).reset_index()


    df_fin = pd.melt(df_fin,
                     id_vars='point_index',
                     value_name='unique_device')

    # check: df_fin[df_fin['point_index']==2914].sort_values(by='hour')

    # add lat lon
    df_fin = pd.merge(df_fin, df_gr, on=['point_index'], how='left')

    dfs = []
    for hour in df_fin['hour'].unique():
        print(hour)
        # get only values for specific hour
        dff = df_fin[df_fin['hour']==hour]
        # blur points = make every point a sum from 6 surrounding points
        blur = 18
        if blur > 0:
            indexes = tree.query(np.deg2rad(dff[['point_lat', 'point_lon']].to_numpy()),
                                 k=blur)[1]
            dff.index = dff['point_index'].astype(int)
            rank_matrix = np.concatenate([dff[['unique_device']].reindex(indexes[:,idx]).to_numpy()
                                          for idx in range(0,blur)],
                                         axis=1)
            dff['unique_device'] = np.sum(rank_matrix, axis=1)

        # save
        dff = dff.reset_index(drop=True)
        dff = dff[['hour', 'point_index', 'point_lat', 'point_lon', 'unique_device']]
        dff.rename(columns={'unique_device': 'metric_traffic'})
        dfs.append(dff)

    df_final = pd.concat(dfs)
    df_final.to_csv('traffic_hours_ts.csv', index=False)
    return None
                    



In [ ]:
    
def create_traffic_weekday_ts(files):
    
    df_agg = read_saved(files)

    # create day column
    df_agg['ts_h'] = pd.to_datetime(df_agg['ts_h'])
    df_agg['weekday'] = df_agg['ts_h'].dt.weekday
    # df_agg['weekday'] = df_agg['ts_h'].dt.weekday


    df_agg = df_agg.groupby(['point_index', 'weekday'])['unique_device'].median().reset_index()


    # add lat lon columns from grid
    df_gr = pd.DataFrame(grid).reset_index()
    df_gr.columns=['point_index', 'point_lat', 'point_lon']
    df_fin = pd.merge(df_gr, df_agg, on=['point_index'], how='left').fillna(0)

    # pivot - unpivot to fill missing weekday values
    df_fin = pd.pivot_table(df_fin,
                            values='unique_device',
                            index='point_index',
                            columns='weekday',
                            fill_value=0).reset_index()


    df_fin = pd.melt(df_fin,
                     id_vars='point_index',
                     value_name='unique_device')

    # check: df_fin[df_fin['point_index']==2914].sort_values(by='weekday')

    # add lat lon
    df_fin = pd.merge(df_fin, df_gr, on=['point_index'], how='left')

    dfs = []
    for weekday in df_fin['weekday'].unique():
        print(weekday)
        # get only values for specific weekday
        dff = df_fin[df_fin['weekday']==weekday]
        # blur points = make every point a sum from 6 surrounding points
        blur = 18
        if blur > 0:
            indexes = tree.query(np.deg2rad(dff[['point_lat', 'point_lon']].to_numpy()),
                                 k=blur)[1]
            dff.index = dff['point_index'].astype(int)
            rank_matrix = np.concatenate([dff[['unique_device']].reindex(indexes[:,idx]).to_numpy()
                                          for idx in range(0,blur)],
                                         axis=1)
            dff['unique_device'] = np.sum(rank_matrix, axis=1)

        # save
        dff = dff.reset_index(drop=True)
        dff = dff[['weekday', 'point_index', 'point_lat', 'point_lon', 'unique_device']]
        dff.rename(columns={'unique_device': 'metric_traffic'})
        dfs.append(dff)

    df_final = pd.concat(dfs)
    df_final.to_csv('traffic_weekday_ts.csv', index=False)
    return None


def create_point_comps():
    competition_data = pd.read_csv('competition_data.csv')
    dfcom = competition_data[['ya_id', 'sub_cat']].groupby(['ya_id']).agg({"sub_cat": list}).reset_index()
    dfcom['sub_cat'] = dfcom['sub_cat'].apply(lambda x: list(set(x)))


    df_gr = pd.DataFrame(grid).reset_index()
    df_gr.columns=['point_index', 'point_lat', 'point_lon']  
    df_agg = competition_data.groupby('point_index').agg({'ya_id': list}).reset_index()

    df_fin = pd.merge(df_gr, df_agg, on=['point_index'], how='left').fillna(0)
    
    def get_unique(m):
        main_lst = []
        for row in m:
            lst = []
            for elem in row:
                if isinstance(elem, list):
                    lst += elem
            main_lst.append(list(set(lst)))

        return main_lst


    def map_id_to_categories(list_id):
        dc = dfcom[dfcom['ya_id'].isin(list_id)].values.tolist()
        newdict = {}
        for key, val in dc:
            for string in val:
                newdict.setdefault(string, []).append(key)

        return newdict

    def map_full(rank_matrix):
        return [map_id_to_categories(row) for row in get_unique(rank_matrix)]


    indexes = tree.query(np.deg2rad(df_fin[['point_lat', 'point_lon']].to_numpy()),
                     k=blur)[1]
    df_fin.index = df_fin['point_index'].astype(int)
    rank_matrix = np.concatenate([df_fin[['ya_id']].reindex(indexes[:,idx]).to_numpy()
                                  for idx in range(0,blur)],
                                 axis=1)
    print('map_full')
    df_fin['ya_id'] = map_full(rank_matrix)
    df_fin = df_fin[['point_index', 'ya_id']].rename(columns={"ya_id": "comps_array"}).reset_index(drop=True)
    df_fin.to_csv('point_comps.csv', index=False)
    return None
    


In [ ]:

def create_point_info():
    dff = pd.read_csv('traffic_weekday_ts.csv')
    dff.sort_values(by=['point_index', 'weekday'], inplace=True)
    dff['unique_device'] = dff['unique_device'].astype(int)

    dff_wd = dff.groupby(['point_index']).agg(
        {'unique_device': list}
    ).reset_index().rename(columns={'unique_device': 'unique_device_weekday'})


    dff = pd.read_csv('traffic_hours_ts.csv')
    dff.sort_values(by=['point_index', 'hour'], inplace=True)
    dff['unique_device'] = dff['unique_device'].astype(int)


    dff_hr = dff.groupby(['point_index']).agg(
        {'unique_device': list}
    ).reset_index().rename(columns={'unique_device': 'unique_device_hour'})
    
    
    df_comps = pd.read_csv('point_comps.csv')
    
    dff_ts = pd.merge(dff_wd, dff_hr, on='point_index', how='left')
    dff_ts = pd.merge(dff_ts, df_comps , on='point_index', how='left')
    
    
    dff_ts.to_csv('point_info.csv', index=False)



In [ ]:
len(geojs)

In [ ]:
point_info = pd.read_csv('point_info.csv')
geojs = geojs
clicked_point = 2674

sub_cat = 'Магазин продуктов'


def make_point_panel(geojs, clicked_point, sub_cat):
    
    mp = geojson_to_df(geojs)
    mp = mp[mp['point_index']==clicked_point]
    pi = point_info[point_info['point_index']==clicked_point]
    
    # get competitors dataframe
    comps = json.loads(pi['comps_array'].values[0].replace("'", '"'))
    comps = comps[sub_cat]
    comps = competition_data[competition_data['ya_id'].isin(comps)].drop_duplicates(subset=['lat', 'long'])
    point_coords = mp['point_lat'].values[0], mp['point_lon'].values[0]
    comps['dist'] = comps.apply(lambda row: haversine_exc((row.lat, row.long), point_coords), axis=1)
    comps['dist'] = comps['dist'].round(5)
    comps['lat'] = comps['lat'].round(5)
    comps['long'] = comps['long'].round(5)
    comps['address'] = comps['address'].str.replace('Россия, Москва,','')
    comps = comps[['name','lat', 'long', 'dist', 'address', 'hours_text']].sort_values(by='dist').reset_index(drop=True)
    comps.columns = ['Название', 'Широта', 'Долгота', 'Расстояние, км', 'Адрес', 'Часы работы']
    
    
    # get traffic_hours timeseries
    ts_hour = json.loads(pi['unique_device_hour'].values[0])
    total = sum(ts_hour)
    ts_hour = pd.DataFrame({str(idx): (hour+1)/(total+1) for idx, hour in enumerate(ts_hour)}, index=[0]).T.reset_index()
    ts_hour.columns = ['Час', 'Уровень траффика (%)']
    
    # create weekly ts
    ts_weekday = json.loads(pi['unique_device_weekday'].values[0])
    total = sum(ts_weekday)
    mapping_wd = {
        0: 'Пн',
        1: 'Вт',
        2: 'Ср',
        3: 'Чт',
        4: 'Пт',
        5:'Сб',
        6:'Вс'
    }
    ts_weekday = pd.DataFrame(
        {mapping_wd[idx]: (weekday+1)/(total+1) for idx, weekday in enumerate(ts_weekday)}, index=[0]).T.reset_index()
    ts_weekday.columns = ['День', 'Уровень траффика (%)']
    
    
    # create common info
    mp_1 = mp[['point_index', 'point_lat', 'point_lon', 'metric_rent', 'metric_traffic', 'count_comps', 'total_rank']]

    mp_1['metric_rent'] = (mp_1['metric_rent']/38).astype(int).astype(str)
    mp_1['metric_traffic'] = mp_1['metric_traffic'].astype(int).astype(str)
    mp_1['point_lat'] = mp_1['point_lat'].round(5).astype(str)
    mp_1['point_lon'] = mp_1['point_lon'].round(5).astype(str)

    mp_1.columns = ['Локация', 'Широта', 'Долгота',
                    'Аренда (руб. в мес. за кв.м.)', 'Траффик (чел. в мес., радиус 0.5 км)',
                    'Конкуренты (шт., радиус 0.5 км)', 'Рейтинг']
#     mp_1 = mp_1.T.reset_index()
#     mp_1.columns = ['Показатель', 'Значение']
    
    
    
    
    # build figure
    
    fig = make_subplots(
        rows=2,
        cols=3,
        column_widths=[0.22, 0.18, 0.6],
        specs=[[{'type': 'table', "rowspan": 2}, {}, {'type': 'table',"rowspan": 2}],
               [{},           {}, {}]],
        subplot_titles=('О локации ', 'Уровень траффика по часам', 'Ближайшие конкуренты', '', 'Уровень траффика по дням')

    )

    trace_general = go.Table(
        #columnwidth = [150,100],
        header=dict(values=list(mp_1.columns),
                    fill_color='#EEF6FF',
                    align='left'),
        cells=dict(values=[mp_1[col] for col in mp_1.columns],
                   fill_color='white',
                   align='left',
                  height=60))


    trace_wd = px.bar(x='День',
           y='Уровень траффика (%)',
           data_frame=ts_weekday)

    trace_wd.update_traces(marker_color='#EEF6FF')
    trace_wd.update_layout(paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')
    trace_wd.update_yaxes(color='white')

    trace_hr = px.bar(x='Час',
           y='Уровень траффика (%)',
           data_frame=ts_hour)

    trace_hr.update_traces(marker_color='#EEF6FF')
    trace_hr.update_layout(paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')
    trace_hr.update_yaxes(color='white')



    trace_comps = go.Table(
        header=dict(values=list(comps.columns),
                    fill_color='#EEF6FF',
                    align='left'),
        cells=dict(values=[comps[col] for col in comps.columns],
                   fill_color='white',
                   align='left'))



    fig.add_trace(trace_general, row=1, col=1)
    fig.add_trace(trace_hr['data'][0], row=1, col=2)
    fig.add_trace(trace_wd['data'][0], row=2, col=2)
    fig.add_trace(trace_comps, row=1, col=3)
    fig.update_layout(width=1300, height=600)

    fig.update_layout(paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')
    fig.update_yaxes(color='white')

    return fig

